Download the data from zenodo and unzip

Pearl River Estuary 
longitude 112-116E
latitude 20.5-23N

In [1]:
import h5py
import torch
import numpy as np
from mpl_toolkits import basemap
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
import cartopy.crs as ccrs
from copy import deepcopy
import h5py
from numpy import meshgrid
import numpy as np
import os

data = h5py.File("../data/raw_data/etopo_2022_modisgrid.mat",'r')
depth = np.array(data['H'])
lon = np.array(data['lon'])
lati = np.array(data['lat'])
chla = []

data_path = "../data/PRE/"
if not os.path.exists(data_path):
    os.makedirs(data_path)

chlas=[]
for i in range(2003,2024):
    data = h5py.File("../data/raw_data/modis_aqua_8d_0m200m_{}.mat".format(str(i)),"r")
    chla = np.array(data['CHLA'])
    tmp = deepcopy(chla[:,1608:1668,7008:7104])
    del chla
    chlas.append(tmp)
    data.close()
    print("Loaded {} year".format(i))



Loaded 2003 year
Loaded 2004 year
Loaded 2005 year
Loaded 2006 year
Loaded 2007 year
Loaded 2008 year
Loaded 2009 year
Loaded 2010 year
Loaded 2011 year
Loaded 2012 year
Loaded 2013 year
Loaded 2014 year
Loaded 2015 year
Loaded 2016 year
Loaded 2017 year
Loaded 2018 year
Loaded 2019 year
Loaded 2020 year
Loaded 2021 year
Loaded 2022 year
Loaded 2023 year


/tmp/ipykernel_2841321/3448380323.py:37: RuntimeWarning: All-NaN slice encountered
  chla_max = np.nanmax(chla,0)
/tmp/ipykernel_2841321/3448380323.py:39: RuntimeWarning: All-NaN slice encountered
  chla_min = np.nanmin(chla,0)
/tmp/ipykernel_2841321/3448380323.py:41: RuntimeWarning: Mean of empty slice
  chla_mean = np.nanmean(chla,0)
/home/mafzhang/miniconda3/envs/torch/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [11]:
data = h5py.File("../data/raw_data/etopo_2022_modisgrid.mat",'r')
depth = np.array(data['H'])
lon = np.array(data['lon'])
lati = np.array(data['lat'])
lon = lon[:,7008:7104].repeat(60,0)
lati = lati[0,1608:1668][:,np.newaxis].repeat(96,1)
chla = np.concatenate(chlas, axis=0)
chla = np.log10(chla)
chla_max = np.nanmax(chla,0)
chla_max = np.nan_to_num(chla_max,nan=0.)
chla_min = np.nanmin(chla,0)
chla_min = np.nan_to_num(chla_min,nan=0.)
chla_mean = np.nanmean(chla,0)
chla_mean = np.nan_to_num(chla_mean,nan=0.)
chla_std = np.nanstd(chla,0)
chla_std = np.nan_to_num(chla_std,nan=1.)
missing_ratio = np.sum(np.isnan(chla),0)/chla.shape[0]
is_sea = missing_ratio<0.9

import torch_geometric
lon_ = lon[is_sea]
lati_ = lati[is_sea]
start = []
end = []
for i in range(is_sea.sum()):
    for j in range(is_sea.sum()):
        if abs(lon_[i]-lon_[j])<0.05 and abs(lati_[i]-lati_[j])<0.05:
            start.append(i)
            end.append(j)
start = torch.Tensor(start)
end = torch.Tensor(end)
adj = torch_geometric.utils.to_dense_adj(torch.stack([start,end],0).long(), torch.zeros(is_sea.sum()).long())

/tmp/ipykernel_2841321/98731427.py:9: RuntimeWarning: All-NaN slice encountered
  chla_max = np.nanmax(chla,0)
/tmp/ipykernel_2841321/98731427.py:11: RuntimeWarning: All-NaN slice encountered
  chla_min = np.nanmin(chla,0)
/tmp/ipykernel_2841321/98731427.py:13: RuntimeWarning: Mean of empty slice
  chla_mean = np.nanmean(chla,0)


In [12]:
np.save(data_path+"chla.npy", 10**chla)
np.save(data_path+"is_sea.npy", is_sea)
np.save(data_path+"max.npy", chla_max)
np.save(data_path+"min.npy", chla_min)
np.save(data_path+"mean.npy", chla_mean)
np.save(data_path+"std.npy", chla_std)


For Chesapeake Bay:
    modify the data_path->"../data/Chesapeake"
    tmp = deepcopy(chla[:,1224:1284,2484:2532])
    lon_ = lon[:,2484:2532].repeat(60,0)
    lati_ = lati[0,1224:1284][:,np.newaxis].repeat(48,1)

For the north of Mexico:
    modify the data_path->"../data/MEXICO"
    tmp = deepcopy(chla[:,1440:1476,2064:2184])
    lon_ = lon[:, 2064:2184].repeat(36,0)
    lati_ = lati[0, 1440:1476][:,np.newaxis].repeat(120,1)

For Yangtze Estuary:
    modify the data_path->"../data/Yangtze"
    tmp = deepcopy(chla[:,1392:1488,7224:7296])
    lon_ = lon[:, 7224:7296].repeat(36,0)
    lati_ = lati[0, 1392:1488][:,np.newaxis].repeat(120,1)
